## Estimator

### TF 编程堆栈

<img src="https://www.tensorflow.org/images/tensorflow_programming_environment.png" width="460px">

我们强烈建议使用下列 API 编写 TensorFlow 程序：

- Estimator：代表一个完整的模型。Estimator API 提供一些方法来训练模型、判断模型的准确率并生成预测。
- Estimator 的数据集：构建数据输入管道。Dataset API 提供一些方法来加载和操作数据，并将数据馈送到模型中。Dataset API 与 Estimator API 合作无间。

### 采用 Estimator 进行编程的概览

Estimator 是 TensorFlow 对完整模型的高级表示。它会处理初始化、日志记录、保存和恢复等细节部分，并具有很多其他功能，以便您可以专注于模型。

Estimator 是从 tf.estimator.Estimator 衍生而来的任何类。TensorFlow 提供一组预创建的 Estimator（例如 LinearRegressor）来实现常见的机器学习算法。

要根据预创建的 Estimator 编写 TensorFlow 程序，您必须执行下列任务：

- 创建一个或多个输入函数。
- 定义模型的特征列。
- 实例化 Estimator，指定特征列和各种超参数。
- 在 Estimator 对象上调用一个或多个方法，传递适当的输入函数作为数据的来源。

我们来看看如何针对鸢尾花分类实施这些任务。

#### 1. 创建输入函数

你必须创建输入函数来提供用于训练、评估和预测的数据。

**输入函数** 是返回 `tf.data.Dataset` 对象的函数，此对象会输出下列两个元素的元组：

- features - Python 字典，其中：
    - 每个键都是特征的名称
    - 每个值都是包含此特征所有值的数组
- label - 包含每个样本的标签值的数组。

下面是输入函数的简单的实现：


In [2]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

输入函数可以以您需要的任何方式生成 `features` 字典和 `label` 列表。不过，建议使用 Dataset API，它可以解析各种数据。概括来讲，Dataset API 包含下列类：

<img src="https://www.tensorflow.org/images/dataset_classes.png" width="400px" />

- Dataset 包含创建和转换数据集的方法和基类。您还可以通过该类从内存中的数据或 Python 生成器初始化数据集。
- TextLineDataset 从文本行中读取记录。
- TFRecordDataset 从 TFRecord 文件中读取记录。
- FixedLengthRecordDataset 从二进制文件中读取具有固定大小的记录。
- Iterator 提供一次访问一个数据集元素的方法。

Dataset API 可以为您处理很多常见情况。例如，使用 Dataset API，您可以轻松地从大量并行文件中读取记录，并将它们合并为单个数据流。

以下是用于程序中进行训练的输入函数。

In [3]:
def train_input_fn(features, labels, batch_size):
    """A input function for training"""
    dataset = tf.data.Dataset.from_tensor_slices((
    dict(features), labels))
    
    # Shuffle, repeat and batch the examples
    return dataset.shuffle(1000).repeat().batch(batch_size)

#### 2. 定义特征列 feature coloumns

**特征列是一个对象，用于说明应该如何使用特征字典中的原始输入数据。**

对于鸢尾花问题，4 个原始特征是数值，因此我们会构建一个特征列的列表，以告知 Estimator 模型将这 4 个特征都表示为 32 位浮点值。

创建特征列的代码如下：

In [7]:
train_x = dict() # stubb
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column_numeric_column(key=key))

#### 3.  实例化 Estimator

鸢尾花问题是一个经典的分类问题。幸运的是，TensorFlow 提供了几个预创建的分类器 Estimator, 其中包括：

- tf.estimator.DNNClassifier：适用于执行多类别分类的深度模型。
- tf.estimator.DNNLinearCombinedClassifier：适用于宽度和深度模型。
- tf.estimator.LinearClassifier：适用于基于线性模型的分类器。

`tf.estimator.DNNClassifier` 似乎是最好的选择。我们将如下所示地实例化 Estimator.


In [9]:
import tensorflow as tf
# Build a DNN with 2 hidden layers and 10 units in each hidden layer
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3
)

W0725 13:50:22.862231 4692223424 estimator.py:1799] Using temporary folder as model directory: /var/folders/1d/n6ds6p0n5hxdp1pfhlr140qc0000gn/T/tmp5h8h_jt5


## 训练、评估和预测

我们已经有一个 Estimator 对象，现在可以调用方法来执行下列操作：
- 训练模型
- 评估经过训练的模型
- 使用经过训练的模型进行预测

### 训练模型

通过调用 Estimator 的 `train` 方法训练模型，如下所示：

```
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(
    train_x, 
    train_y, 
    args.batch_size
    ),
    steps=args.train_steps)
```

### 评估经过训练的模型

调用 `evaluate` 方法进行预测。

```
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(test_x, test_y, args.batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
```

### 复用经过训练的模型进行预测（推理）

```
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            batch_size=args.batch_size))
```